### Quickstart : Compare runs, choose a model, and deploy it to a REST API. 

In this quickstart : 

1. Run a hyperparameter sweep on a training script. 
2. Compare the results of runs in the MLFlow UI. 
3. Choose the best run and register it as a model. 
4. Deploy the model to a rest api. 
5. Build a container image suitable for deployment to a cloud platform. 


In [8]:
## Import libraries 

import keras 
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 

import mlflow 
from mlflow.models import infer_signature

In [9]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [12]:
## Split the data into training validation and test 

train, test = train_test_split(data, test_size=0.20, random_state=42)


In [13]:
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4665,7.3,0.17,0.36,8.20,0.028,44.0,111.0,0.99272,3.14,0.41,12.4,6
1943,6.3,0.25,0.44,11.60,0.041,48.0,195.0,0.99680,3.18,0.52,9.5,5
3399,5.6,0.32,0.33,7.40,0.037,25.0,95.0,0.99268,3.25,0.49,11.1,6
843,6.9,0.19,0.35,1.70,0.036,33.0,101.0,0.99315,3.21,0.54,10.8,7
2580,7.7,0.30,0.26,18.95,0.053,36.0,174.0,0.99976,3.20,0.50,10.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [16]:
## Training Data 
train_x = train.drop(columns=['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

## Validation Data 
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

## Test set 
test_x = test.drop(columns=['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

signature = infer_signature(train_x, train_y)


In [23]:
## ANN Model Creation 

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    ## Define model architecture 
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model=keras.Sequential(
        [
            keras.Input(shape=(train_x.shape[1], ) ), 
            keras.layers.Normalization(mean=mean, variance=var), 
            keras.layers.Dense(64, activation='relu'), 
            keras.layers.Dense(1)
        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'], 
        momentum=params['momentum'], 
    ), 
    loss="mean_squared_error", 
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN Model with lr and momentum parameters with mlflow tracking 
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, 
          validation_data=(valid_x, valid_y),  # Ensure correct tuple format
          epochs=epochs, 
          batch_size=64)
        
        ## Evaluate model 
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse = eval_result[1]

        ## Log the parameters and results 
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {
            "loss": eval_rmse, "status": STATUS_OK, "model": model
        }



In [19]:
def objective(params):
    ## MLFlow will track the params and results for each run 
    result = train_model(
        params, 
        epochs=3, 
        train_x=train_x, 
        train_y=train_y, 
        valid_x=val_x, 
        valid_y=val_y, 
        test_x=test_x, 
        test_y=test_y
    )
    return result

In [20]:
space={
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 1.0) 
}

In [24]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    ## Hyperparameter search using hyperopt 
    trials= Trials()
    best= fmin(
        fn=objective, 
        space=space, 
        algo=tpe.suggest,
        max_evals=4, 
        trials=trials
    )

    ## Fetch details of the best run 
    best_run = sorted(trials.results, key=lambda x:x["loss"])[0]

    ## Log the best parameters, loss and model 
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'], 'model', signature=signature)

    ## Print out best parameters and corresponding loss 
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")
    

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

job exception: TensorFlowTrainer.fit() got multiple values for argument 'batch_size'



  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]


TypeError: TensorFlowTrainer.fit() got multiple values for argument 'batch_size'